# Import Bibliotheken

In [1]:
# Festlegung des Device
import platform

# Laden der Daten
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Operationen mit Texten
import torchtext
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import spacy

# Modell
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

# Evaluierung
import torchmetrics

# Auswertung der Evaluation
import pandas as pd
import altair as alt
import numpy as np

from sklearn import preprocessing

### Package Versionen

In [2]:
for package in [torchtext,torch, torchmetrics, spacy, alt, pd]:
    print(f'{package.__name__}: {package.__version__}')

torchtext: 0.13.1
torch: 1.12.1
torchmetrics: 0.9.3
spacy: 3.4.2
altair: 4.2.0
pandas: 1.4.3


# Device Auswahl

In [3]:
device = torch.device(
    "cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Daten

## Dataset

### Erstellung Dataset

In [4]:
comment_df = pd.read_excel('../../data/clean/Google_Rezensionen.xlsx')
comment_df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


In [5]:
class TeamGoogleBewertungenDataSet(Dataset):

    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        text = comment_df.iloc[index]['Kommentar']
        category = comment_df.iloc[index]['Kategorien']
        return text, category

### Bestimmung Test- und Traningsdaten

In [6]:
# Bestimmung der Länge der Test- und Traningsdaten
dataset = TeamGoogleBewertungenDataSet(comment_df)
len_train = round(len(dataset)* 0.7)
len_test = round(len(dataset) * 0.3)
assert len(comment_df) == len_train + len_test

In [7]:
# Zufällige Aufsplittung der Anzahl von Test- und Trainingsdaten
train_set, test_set = torch.utils.data.random_split(dataset, [len_train, len_test]) 

## Dataloader

### Tokenisierung

In [9]:
tokenizer = get_tokenizer(
    'spacy', 
    language='de_core_news_lg')

### Erstellung Vokabular

In [11]:
def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)

In [12]:
# Länge des Trainungsets
len(train_set)

2798

In [13]:
tokens = yield_tokens(train_set)

In [14]:
next(iter(tokens))

['einfach', 'auf', 'günstige', 'preise', 'achten', 'lohnt', 'sich']

In [15]:
vocab = build_vocab_from_iterator(tokens, min_freq=2, specials=["<unk>"])

vocab.set_default_index(vocab["<unk>"])

In [16]:
len(vocab)

2376

In [18]:
vocab.lookup_token(0)

'<unk>'

### Encoding

In [19]:
onehot = torch.zeros(1,len(vocab))

In [21]:
def encode(text, vocab):
    tokens = tokenizer(text)
    onehot = torch.zeros(1,len(vocab))
    onehot[:,vocab(tokens)] = 1
    return onehot

### Multilabel Encoding

In [23]:
df = comment_df.explode('Kategorien')
df

,Unnamed: 0,Tankstellenname,Datum,Bewertung,Kommentar,Kategorien
0,4,TS Schleswig MC,2022-09-30 06:46:42,5,super nettes personal gutes frühstück mit groß...,"Personal,Shop,Bistro"
1,5,TS Prisdorf,2022-09-29 17:22:44,5,sehr höfliche bedienstete,Personal
2,6,TS Wanderup,2022-09-29 15:38:02,5,immer gerne dort nettes und zuvorkommendes per...,"Personal,Waschanlage"
3,10,TS Handewitt,2022-09-28 17:05:28,5,nettes freundliches personal,Personal
4,12,TS Bremen,2022-09-28 04:24:34,4,normale tankstelle die aber in der tank app ni...,"Pricing,DigitalFueling"
...,...,...,...,...,...,...
3992,11072,TS Jübek,2015-04-07 17:45:02,5,dies ist einfach die beste tankstelle die ich ...,"Kraftstoffauswahl,Waschanlage"
3993,11077,TS Jübek,2014-07-16 07:08:37,5,sehr gut günstig guter shop und einige andere ...,"Pricing,Shop,Kraftstoffauswahl,Waschanlage,Sta..."
3994,11079,TS Neustadt am Rüb.,2013-09-21 15:26:28,5,öffnungszeiten mo so 06 00 22 00 uhr,Öffnungszeiten
3995,11080,TS Handewitt(SP),2012-11-05 11:16:00,5,genug platz fuer wohnwagen rechts beim autogas...,"Personal,Kraftstoffauswahl"


#### Kategorien splitten

In [24]:
einzelLabels = comment_df['Kategorien'].str.split(',')
label_list_export = einzelLabels.to_list()
label_list_export

[['Personal', 'Shop', 'Bistro'],
 ['Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal'],
 ['Pricing', 'DigitalFueling'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing', 'Kraftstoffauswahl'],
 ['Personal', 'Öffnungszeiten'],
 ['Personal', 'SB-Waschboxen', 'Waschanlage'],
 ['Shop'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal', 'Waschanlage'],
 ['Personal', 'Pricing'],
 ['Personal', 'Pricing'],
 ['Bistro'],
 ['Personal', 'Pricing', 'Shop', 'Waschanlage'],
 ['Personal'],
 ['Personal'],
 ['Bistro'],
 ['Erscheinungsbild'],
 ['Personal', 'Pricing'],
 ['Personal'],
 ['Personal'],
 ['Personal', 'Shop'],
 ['Erscheinungsbild'],
 ['Personal', 'Bistro'],
 ['Erscheinungsbild', 'Nacht-/Tankautomat', 'Waschanlage', 'Staubsauger'],
 ['Pricing'],
 ['Personal'],
 ['Erscheinungsbild', 'Personal'],
 ['Personal'],
 ['Bistro'],
 ['Pricing', 'Kraftstoffauswahl'],
 ['Pricing'],
 ['Nacht-/Tankautomat'],
 ['Shop', 'Bistro'],
 ['Erscheinungsbild', 'Personal', 'Sanitär

##### In der nachfolgenden Zelle wird aus der verschachtelten Liste eine "flache Liste"

In [25]:
flat_list = []
for sublist in label_list_export:
    for item in sublist:
        flat_list.append(item)

flat_list

['Personal',
 'Shop',
 'Bistro',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'DigitalFueling',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Kraftstoffauswahl',
 'Personal',
 'Öffnungszeiten',
 'Personal',
 'SB-Waschboxen',
 'Waschanlage',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Waschanlage',
 'Personal',
 'Pricing',
 'Personal',
 'Pricing',
 'Bistro',
 'Personal',
 'Pricing',
 'Shop',
 'Waschanlage',
 'Personal',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Pricing',
 'Personal',
 'Personal',
 'Personal',
 'Shop',
 'Erscheinungsbild',
 'Personal',
 'Bistro',
 'Erscheinungsbild',
 'Nacht-/Tankautomat',
 'Waschanlage',
 'Staubsauger',
 'Pricing',
 'Personal',
 'Erscheinungsbild',
 'Personal',
 'Personal',
 'Bistro',
 'Pricing',
 'Kraftstoffauswahl',
 'Pricing',
 'Nacht-/Tankautomat',
 'Shop',
 'Bistro',
 'Erscheinungsbild',
 'Personal',
 'Sanitär',
 'Bistro',
 'Pricing',
 'Verkehrsanbindung'

##### Doppelte Werte werden entfernt, so dass jeder Eintrag nur einmalig vorhanden ist. Dies erreichen wir mit set()

In [26]:
flat_set = set(flat_list)
flat_set

{'AdBlue',
 'Bistro',
 'DigitalFueling',
 'E-Fuels',
 'E-Mobilität',
 'Erscheinungsbild',
 'Kartenakzeptanzen',
 'Kraftstoffauswahl',
 'Luft',
 'Nacht-/Tankautomat',
 'Paketservice',
 'Personal',
 'Pricing',
 'SB-Waschboxen',
 'Sanitär',
 'Shop',
 'Staubsauger',
 'Tankpool',
 'Verkehrsanbindung',
 'WLAN',
 'Waschanlage',
 'Werkstatt',
 'Zapfsäulen',
 'Öffnungszeiten'}

In [27]:
unflat = [[x] for x in flat_set]

In [28]:
label_vocab = build_vocab_from_iterator(unflat, min_freq=1, specials=["<unk>"])

label_vocab.set_default_index(vocab["<unk>"])

In [29]:
label_vocab.lookup_token(3)

'DigitalFueling'

#### Dictionary erstellen

In [30]:
label_dict = {label:i for i, label in enumerate(flat_set)}
label_dict

{'WLAN': 0,
 'Sanitär': 1,
 'Öffnungszeiten': 2,
 'Paketservice': 3,
 'E-Fuels': 4,
 'Kartenakzeptanzen': 5,
 'Luft': 6,
 'Waschanlage': 7,
 'Personal': 8,
 'Tankpool': 9,
 'Werkstatt': 10,
 'SB-Waschboxen': 11,
 'Erscheinungsbild': 12,
 'DigitalFueling': 13,
 'Zapfsäulen': 14,
 'AdBlue': 15,
 'Bistro': 16,
 'Shop': 17,
 'Staubsauger': 18,
 'Verkehrsanbindung': 19,
 'Nacht-/Tankautomat': 20,
 'E-Mobilität': 21,
 'Pricing': 22,
 'Kraftstoffauswahl': 23}

### Erstellung Dataloader

In [38]:
def collate_batch(batch):
    label_list, text_list = [], []
 
    for (_text,_label) in batch:
    
        # Vorverarbeitung der Label
        processed_labels = encode(_label, label_vocab)

        label_list.append(processed_labels)

        # Vorverarbeitung der Texte
        processed_text = encode(_text, vocab)
        
        # Zusammenführen sämtlicher Textrepräsentationen in einer Liste
        text_list.append(processed_text)
 
    # Zusammenführen aller Label in einem Tensor
    labels = torch.cat(label_list, dim=0)
    
    # Verbinden der Tensoren in text_list zu einem Tensor
    texts = torch.cat(text_list, dim = 0)

    # Ausgabe der Texte und der Label
    return texts.to(device), labels.to(device)

In [39]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, collate_fn=collate_batch, num_workers=0)

## Architektur und Training

### Architektur

In [40]:
class LinearTextClassificationModel(nn.Module):

    def __init__(self, vocab_size, num_class):
        super(LinearTextClassificationModel, self).__init__()
        self.fc1 = nn.Linear(vocab_size, 200)
        self.fc2 = nn.Linear(200, num_class)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.softmax(x, dim=1)
        return output

In [41]:
vocab_size = len(vocab)
num_class = 25
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

### Training

In [42]:
# Hyperparameter

## Festlegung Lernrate
learning_rate = 0.05

## Initialisierung Fehlerfunktion
loss_fn = nn.MSELoss()

## Initialisierung Optimizer
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

## Definition der Epochen
num_epochs = 50

In [43]:
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)

loss_hist = {}
accuracy_hist = {}

In [44]:
for (text, label) in train_loader:
    labelnew = label.type(torch.LongTensor)
    print(labelnew[[0][0]])

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0])
tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0])
tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,


In [45]:
# Training
model.train()

for epoch in range(num_epochs):

    # Dokumentation Loss -> Erkennung ob Netz konvergiert
    running_loss = 0.0
    num_batches = 0

    for (text, label) in train_loader:
        num_batches += 1
        pred = model(text)

        # Der Fehler wird berechnet
        loss = loss_fn(pred, label)
        # Der Fehler wird über das Netz zurückpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        ## Bestimmung der Accuracy für den Batch
        train_accuracy(pred, label.type(torch.LongTensor))
        
        # running loss
        running_loss +=loss.item()
 
    loss_hist[epoch] = running_loss/num_batches
 
    batch_train_accuracy = train_accuracy.compute()
    print(f"Training Accuracy for epoch {epoch}:{batch_train_accuracy}")
    accuracy_hist[epoch] = batch_train_accuracy.cpu().numpy()
    train_accuracy.reset()

Training Accuracy for epoch 0:0.30307361483573914
Training Accuracy for epoch 1:0.4045746922492981
Training Accuracy for epoch 2:0.43566834926605225
Training Accuracy for epoch 3:0.45032164454460144
Training Accuracy for epoch 4:0.4614010155200958
Training Accuracy for epoch 5:0.466761976480484
Training Accuracy for epoch 6:0.4714081585407257
Training Accuracy for epoch 7:0.4724803566932678
Training Accuracy for epoch 8:0.47462472319602966
Training Accuracy for epoch 9:0.4796282947063446
Training Accuracy for epoch 10:0.47998571395874023
Training Accuracy for epoch 11:0.48391708731651306
Training Accuracy for epoch 12:0.4842744767665863
Training Accuracy for epoch 13:0.4849892854690552
Training Accuracy for epoch 14:0.48463186621665955
Training Accuracy for epoch 15:0.48463186621665955
Training Accuracy for epoch 16:0.48284488916397095
Training Accuracy for epoch 17:0.48463186621665955
Training Accuracy for epoch 18:0.4860614836215973
Training Accuracy for epoch 19:0.48856326937675476


In [46]:
total_train_accuracy = train_accuracy.compute()

c:\Users\flori\anaconda3\envs\ki_env\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: The ``compute`` method of metric Accuracy was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)


### Auswertung Fehlerentwicklung

In [47]:
loss_df = pd.DataFrame.from_dict(loss_hist, orient= 'index').reset_index()

loss_df.columns = ['Epoch', 'Loss']
loss_df.head()

,Epoch,Loss
0,0,0.063809
1,1,0.053275
2,2,0.050271
3,3,0.049804
4,4,0.048862


In [48]:
train_chart = alt.Chart(loss_df).mark_line().encode(
    x=alt.X('Epoch', title = 'Anzahl Epochen'),
    y=alt.Y('Loss', title = 'Mittlerer Fehler')
)
train_chart

alt.Chart(...)

### Auswertung Genauigkeit

In [49]:
accuracy_df = pd.DataFrame.from_dict(accuracy_hist, orient = 'index').reset_index()
accuracy_df.columns = ['Epoch', 'Accuracy']
accuracy_df.head()

,Epoch,Accuracy
0,0,0.303074
1,1,0.404575
2,2,0.435668
3,3,0.450322
4,4,0.461401


In [50]:
accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x=alt.X('Epoch',title = 'Anzahl Epochen'),
    y=alt.Y('Accuracy', title = 'Genauigkeit')
)
accuracy_chart

alt.Chart(...)

## Evaluierung

In [51]:
model.eval()

LinearTextClassificationModel(
  (fc1): Linear(in_features=2376, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=25, bias=True)
)

### Gesamtevaluierung

In [52]:
test_loader = DataLoader(test_set, batch_size= 64, shuffle=True,collate_fn=collate_batch)

In [53]:
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)

with torch.no_grad():
    for (text, label) in test_loader:
        # Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()

In [54]:
total_valid_accuracy

tensor(0.3978)

### Evaluierung nach Klassen

In [55]:
# Evaluieren
valid_accuracy = torchmetrics.Accuracy(
    average=None,
    num_classes=25
).to(device)

with torch.no_grad():
    for(text, label) in test_loader:
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        valid_accuracy.update(pred, label.type(torch.LongTensor))
    total_valid_accuracy = valid_accuracy.compute()
    total_valid_accuracy

In [56]:
total_valid_accuracy

tensor([0.5663, 0.9892, 0.8432, 0.9942,    nan, 0.9975, 0.8657, 0.9608, 0.9291,
        0.9833, 0.9725, 0.9900, 0.6814, 0.8198, 0.9900, 0.9583, 0.8791, 0.9925,
        0.9983, 0.9691,    nan, 0.8924, 0.9942, 0.9917, 0.9625])

### Evaluierung nach der Anzahl der Epochen

In [57]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=True, collate_fn=collate_batch)

In [58]:
test_loader = DataLoader(test_set, batch_size=64, shuffle=True, collate_fn=collate_batch)

In [59]:
vocab_size = len(vocab)
num_class = 25
model = LinearTextClassificationModel(vocab_size, num_class).to(device)

In [60]:
# Hyperparameter

## Festlegung Lernrate
learning_rate = 0.05

## Initialisierung Fehlerfunktion
loss_fn = nn.MSELoss()

## Initialisierung Optimizer
optimizer = torch.optim.Adam(model.parameters(),lr = learning_rate)

## Definition der Epochen
num_epochs = 50

In [61]:
# Evaluierung wurd vorbereitet
train_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)
valid_accuracy = torchmetrics.Accuracy(subset_accuracy=True,threshold=0.6).to(device)

train_accuracy_hist = {}
train_loss_hist = {}

valid_accuracy_hist = {}
valid_loss_hist = {}

In [62]:
for epoch in range(num_epochs):

    # Loss wird dokumentiert, sodass zu erkenne ist, ob das Netz konvergiert
    num_train_batches = 0
    num_eval_batches = 0
    train_running_loss = 0.0
    eval_running_loss = 0.0

    model.train()
    for (text, label) in train_loader:
        num_train_batches += 1
        # Die Vorhersage wird für das Model erzeugt
        pred = model(text)
        # Der Fehler wird berechnet
        # Als Ausgabe wird ein Tensor zurückgegeben
        loss = loss_fn(pred, label)
        # Der Fehler wird über das Netz zurükpropagiert
        loss.backward()
        # Die Gewichte werden angepasst
        optimizer.step()
        # Gradienten zurücksetzen
        optimizer.zero_grad()
        # Aktualisierung des States der train_accurracy
        train_accuracy.update(pred, label.type(torch.LongTensor))
        # running loss
        train_running_loss += loss.item()

    train_loss_hist[epoch] = train_running_loss / num_train_batches

    # Berechnung der Trainingsgenauigkeit für die Epoche
    total_train_accuracy = train_accuracy.compute()
    # Hinzufügen der Trainingsgenauigkeit zu dem Dictionary
    train_accuracy_hist[epoch] = total_train_accuracy.cpu().numpy()

    # Validierung
    model.eval()
    with torch.no_grad():
        for (text, label) in test_loader:
            num_eval_batches += 1
            # Die Vorhersage wird für das Model erzeugt
            pred = model(text)
            # Test loss
            eval_loss = loss_fn(pred, label)
            # Running loss
            eval_running_loss += eval_loss.item()
            # Aktualisierung des States der valid_accuracy
            valid_accuracy.update(pred, label.type(torch.LongTensor))
    
    valid_loss_hist[epoch] = eval_running_loss/num_eval_batches

    # Berechnung der Validierungsgenauigkeit für die Epoche
    total_valid_accuracy = valid_accuracy.compute()
    # Hinzufügen der Validierungsgenauigkeit zum Dictionary
    valid_accuracy_hist[epoch] = total_valid_accuracy.cpu().numpy()

    print(f"Training Accuracy for epoch {epoch}: {total_train_accuracy}")
    print(f"Validation Accuracy for epoch {epoch}: {total_valid_accuracy}")
    train_accuracy.reset()
    valid_accuracy.reset()

Training Accuracy for epoch 0: 0.3238027095794678
Validation Accuracy for epoch 0: 0.4017155170440674
Training Accuracy for epoch 1: 0.40957826375961304
Validation Accuracy for epoch 1: 0.4453180730342865
Training Accuracy for epoch 2: 0.44496068358421326
Validation Accuracy for epoch 2: 0.46354538202285767
Training Accuracy for epoch 3: 0.45782703161239624
Validation Accuracy for epoch 3: 0.47355252504348755
Training Accuracy for epoch 4: 0.4724803566932678
Validation Accuracy for epoch 4: 0.47712650895118713
Training Accuracy for epoch 5: 0.47355252504348755
Validation Accuracy for epoch 5: 0.47998571395874023
Training Accuracy for epoch 6: 0.47712650895118713
Validation Accuracy for epoch 6: 0.4853466749191284
Training Accuracy for epoch 7: 0.4803431034088135
Validation Accuracy for epoch 7: 0.48463186621665955
Training Accuracy for epoch 8: 0.4832022786140442
Validation Accuracy for epoch 8: 0.48641887307167053
Training Accuracy for epoch 9: 0.48570406436920166
Validation Accuracy 

In [63]:
train_loss_df = pd.DataFrame.from_dict(train_loss_hist, orient = "index").reset_index()
train_loss_df["type"] = "Train"
valid_loss_df = pd.DataFrame.from_dict(valid_loss_hist, orient = "index").reset_index()
valid_loss_df["type"] = "Test"
loss_df = pd.concat([train_loss_df,valid_loss_df])
loss_df.columns = ["Epoch", "Loss", "Type"]

In [64]:
train_eval_loss_chart = alt.Chart(loss_df).mark_line().encode(
    x = alt.X('Epoch',title='Anzahl Epochen'),
    y = alt.Y('Loss',title='Mittlerer Fehler'),
    color = alt.Color('Type',title='Typ')
)
train_eval_loss_chart

alt.Chart(...)

In [65]:
valid_accuracy_hist

{0: array(0.40171552, dtype=float32),
 1: array(0.44531807, dtype=float32),
 2: array(0.46354538, dtype=float32),
 3: array(0.47355253, dtype=float32),
 4: array(0.4771265, dtype=float32),
 5: array(0.4799857, dtype=float32),
 6: array(0.48534667, dtype=float32),
 7: array(0.48463187, dtype=float32),
 8: array(0.48641887, dtype=float32),
 9: array(0.48713368, dtype=float32),
 10: array(0.48927805, dtype=float32),
 11: array(0.48927805, dtype=float32),
 12: array(0.49106506, dtype=float32),
 13: array(0.48927805, dtype=float32),
 14: array(0.49070764, dtype=float32),
 15: array(0.48892066, dtype=float32),
 16: array(0.48999286, dtype=float32),
 17: array(0.49070764, dtype=float32),
 18: array(0.49213725, dtype=float32),
 19: array(0.49177983, dtype=float32),
 20: array(0.49356684, dtype=float32),
 21: array(0.49285203, dtype=float32),
 22: array(0.49249464, dtype=float32),
 23: array(0.49392423, dtype=float32),
 24: array(0.49249464, dtype=float32),
 25: array(0.49320942, dtype=float32)

In [66]:
train_accuracy_hist

{0: array([0.5663053 , 0.9891576 , 0.84320265, 0.9941618 ,        nan,
        0.9974979 , 0.86572146, 0.96080065, 0.9291076 , 0.98331946,
        0.9724771 , 0.98999166, 0.6814012 , 0.81984985, 0.98999166,
        0.95829856, 0.8790659 , 0.99249375, 0.99833196, 0.96914095,
               nan, 0.89241034, 0.9941618 , 0.9916597 , 0.96246874],
       dtype=float32),
 1: array(0.40171552, dtype=float32),
 2: array(0.44531807, dtype=float32),
 3: array(0.46354538, dtype=float32),
 4: array(0.47355253, dtype=float32),
 5: array(0.4771265, dtype=float32),
 6: array(0.4799857, dtype=float32),
 7: array(0.48534667, dtype=float32),
 8: array(0.48463187, dtype=float32),
 9: array(0.48641887, dtype=float32),
 10: array(0.48713368, dtype=float32),
 11: array(0.48927805, dtype=float32),
 12: array(0.48927805, dtype=float32),
 13: array(0.49106506, dtype=float32),
 14: array(0.48927805, dtype=float32),
 15: array(0.49070764, dtype=float32),
 16: array(0.48892066, dtype=float32),
 17: array(0.4899928

In [67]:
# Erste Epoche löschen, da diese nicht neu initialisiert wird und Daten aus vorherigen Schritten enthält

del train_accuracy_hist[0]
del valid_accuracy_hist[0]

In [68]:
train_accuracy_df = pd.DataFrame.from_dict(train_accuracy_hist, orient = "index").reset_index()
train_accuracy_df["type"] = "Train"
valid_accuracy_df = pd.DataFrame.from_dict(valid_accuracy_hist, orient = "index").reset_index()
valid_accuracy_df["type"] = "Test"
accuracy_df = pd.concat([train_accuracy_df,valid_accuracy_df])
accuracy_df.columns = ["Epoch", "Accuracy", "Type"]

In [69]:
accuracy_df

,Epoch,Accuracy,Type
0,1,0.401716,Train
1,2,0.445318,Train
2,3,0.463545,Train
3,4,0.473553,Train
4,5,0.477127,Train
...,...,...,...
44,45,0.493924,Test
45,46,0.494282,Test
46,47,0.492852,Test
47,48,0.492495,Test


In [70]:
train_eval_accuracy_chart = alt.Chart(accuracy_df).mark_line().encode(
    x = alt.X('Epoch',title='Anzahl Epochen'),
    y = alt.Y('Accuracy',title='Genauigkeit'),
    color = alt.Color('Type',title='Typ')
)
train_eval_accuracy_chart

alt.Chart(...)